# Лит-компилятор

Этот ноутбук позволяет сгенерировать лит-компилятор, который добавляет "вкусностей" (см. ниже). Лит-компилятор преобразует ноутбук из высокоуровнего "литературного" формата в формат Ассемблера.

Технически ноутбук предназначен для обработки ассемблером, поэтому в нем используется формат Ассемблера.

## Общая идея

К возможностям собственно "литературного" программирования (а это - Ассемблер) добавились "вкусности":

**Дублирование**

Очень не хочется дублировать одно и то же (имя функции, параметры и их описание, результат...) - раз уж мы "генерируем" код, почему бы себе такого не позволить? См. пример литературного ноутбука (lit_smoke.ipynb) и юнит-тесты здесь

**Док-строки**

Документация содержится в литературном ноутбуке - в модуль она не "переезжает". В модуль переносятся только кодовые ячейки и только те из них, которые помечены соответствующим тэгом (список и семантику тэгов нужно смотреть по коду или в примерах - директория `Examples`). 

**Фолдируемость**

Юпитер лаб позволяет замечательно визуально организовать код, если использовать заголовки (см. пример литературного ноутбука).

## Отладка

Отлаживаемся, естественно, прямо здесь:

* ячейка ниже (через ASM) генерирует два файла
    * lit.py: модуль, обеспечивающий возможность обработки литературных ноутбуков (преобразования их в ASM ноутбуки)
    * test_lit.py: тесты для этого модуля
* при запуске тестов (из test_lit.py)
    * провевряем, что ничего не сломали 
    * один из тестов обрабатывает литературный тестовый ноутбук: lit.genAsmNotebook("../Examples/lit_smoke.ipynb")
        * в результате получается ASM ноутбук (lit_smoke_asm.ipynb) - дополнительный "интеграционный" тест
        * его можно ASM-ом преобразовать в модуль (lit_smoke.py)

In [1]:
import asm

In [4]:
asm.processFile("lit_asm.ipynb")
asm.processFile("lit_asm.ipynb","test")
!pytest test_lit.py

============================= test session starts ==============================
platform linux -- Python 3.11.8, pytest-8.1.1, pluggy-1.4.0
rootdir: /home/korolmi/Work/GitSrc/lit_notebooks/lit_notebooks/Src
plugins: anyio-4.3.0
collected 10 items                                                             

test_lit.py ..........                                                   [100%]

============================== 10 passed in 0.06s ==============================


## Код (в ассемблерном виде)

Функции `procXXX` формируют пары (тэг, список строк) = заготовки для кодовых ячеек ASM ноутбука.

Функциям передается

* содержимое ячейки (то, что в "source" ноутбука)
* тэг (вдруг обработка зависит от этого)
* "контекст" (**kwargs) - содержимое контекста
    * возникло из необходимости знать имя текущей функции
    * может помочь в делании опциональным тэга return 

Обработка литературного ноутбука будет происходить в два этапа

* asmNotebook = genAsmNotebook(litNotebook): генерация ASM ноутбука
* asm.processFile(asmNoteebook)
* asm.processFile(asmNoteebook, "test")

Список поддерживаемых (обрабатываемых) тэгов - см. функцию `genAsmNotebook()`.

### _getFuncName: utility functions

Выдирает имя функции из ячейки с тэгом `function` (см. описание формата и примеры в procFuncTag() ниже)

#### Source

In [ ]:
def _getFuncName(cellContent):

    # разбирает содержимое
    funcName = cellContent[0].split("## ")[1] # предполагаем, что может быть заголовком уровня 2 и выше
    funcName = funcName.split("::")[0] # возможен комментарий, его убираем
    return funcName.strip() # убираем пробелы, если есть

#### Tests

In [ ]:
def test_getFuncName_smoke():

    assert lit._getFuncName(["#### GenSomething:: Эта функция делает что-то..."])=="GenSomething"

### procFuncTag

Функция обработки ячейки с заголовком 

* класса
* функции
* метода
* теста

Предполагаемый формат

    {Решетки} {имя функции или класса} [:: {описание функции или класса}]

где

* решетки: маркдаун заголовок уровня 2+
* имя (функции)
* описание (функции): должно помещаться на одну строку
* лексемы разделяются пробелами
* описание игнорируется
  
Пример

    #### GenSomething:: Эта функция делает что-то... (описание функции)
    
Результат

для функции

    def GenSomething(

и возвращается тэг mod

для класса

    class GenSomething:

и возвращается тэг mod

для метода

        def GenSomething(self,

и возвращается тэг mod

для теста

    def test_CurrentFunction_GenSomething(

и возвращается тэг test

Имя текущей функции передается через параметр `func` контекста (и должно присутствовать в контексте)

#### Source

In [ ]:
def procFuncTag(cellContent, tag, func, **kwargs):

    # результат
    match tag:
        case "function":
            return ( "mod", [f"""def {_getFuncName(cellContent)}("""] )
        case "class":
            return ( "mod", [f"""class {_getFuncName(cellContent)}:"""] )
        case "method":
            return ( "mod", [f"""    def {_getFuncName(cellContent)}(self,"""] )
        case "testdef":
            return ( "test", [f"""def test_{func}_{_getFuncName(cellContent)}("""] )

#### Tests

In [ ]:
def test_procFuncTag_smoke():

    assert lit.procFuncTag(["#### GenSomething:: Эта функция делает что-то..."],tag="function",func="GenSomething")==("mod",["def GenSomething("])

In [ ]:
def test_procFuncTag_test_smoke():

    assert lit.procFuncTag(["#### GenSomething:: Этот тест проверяет что-то..."],tag="testdef",func="FUNC")==("test",["def test_FUNC_GenSomething("])

### procParamTag

Обработка параметра функции или теста

Предполагаемый формат

    {имя параметра} [:тип] [= начальное значение] [:: комментарий]

Где

* все, кроме имени параметра, опционально, но должно поместиться на одну строку

Важно: до "комментария" сохранен синтаксис питона, поэтому все просто переносится как есть.

Пример

    numInst: int = 4:: сколько штук нужно сгенерировать
    
Результат

    {индент}numInst: int = 4, 

#### Source

In [ ]:
def procParamTag(cellContent, tag, **kwargs):

    # numInst -> int = 4:: сколько штук нужно сгенерировать
    
    # разбирает содержимое
    parStr = cellContent[0].split("::")[0] # возможен комментарий, его убираем

    # результат
    return ( "test" if tag=="testparam" else "mod", [f"""    {parStr},"""] )

#### Tests

In [4]:
def test_procParamTag_smoke():

    assert lit.procParamTag(["numInst: int = 4:: сколько штук нужно сгенерировать"],tag="param")==("mod",["    numInst: int = 4,"])

In [4]:
def test_procParamTag_for_test():

    assert lit.procParamTag(["numInst: int = 4:: сколько штук нужно сгенерировать"],tag="testparam")==("test",["    numInst: int = 4,"])

### procReturnTag

Обработка результата функции или метода

Предполагаемый формат

    [тип] [:: комментарий]

Где

* все опционально, но должно поместиться на одну строку

Если не хочется описывать результат, то можно оставить пустую строку с тэгом `return` или не использовать его вообще...

Пример

    int:: вычисленный факториал числа
    
Результат

    ) -> int: 

В случае обработки метода (берем из контекста) добавляется индент.

#### Source

In [ ]:
def procReturnTag(cellContent, tag, isMethod, **kwargs):

    # int:: вычисленный факториал числа
    
    # разбирает содержимое
    retStr = cellContent[0].split("::")[0].strip() # возможен комментарий, его убираем

    if len(retStr)>0:
        retStr = f" -> {retStr}"
        
    # результат
    indent = ""
    if isMethod:
        indent = "    "
    return ( "mod", [f"""{indent}){retStr}:"""] )

#### Tests

In [4]:
def test_procReturnTag_smoke():

    assert lit.procReturnTag(["int:: вычисленный факториал числа"],tag="return",isMethod=False)==("mod",[") -> int:"])

### procBodyTag

Обработка тела функции, метода, теста или блока кода верхнего уровня

Предполагаемый формат

    {тело функции}
    
Важно: все просто переносится как есть, каждая строка индентируется.

Пример

    print(123)
    print(4)
    
Результат

    {индент}print(123)
    {индент}print(4)

Для блока кода (тэг `block`) индент отсутствует.

Для тела метода добавляется дополнительный индент.

**В отсутствие тэга return**

Необходимо добавить закрывающую скобку и двоеточие - корректно завершить определение функции.

**Для тестов**

Необходимо возвращать тэг test

#### Source

In [ ]:
def procBodyTag(cellContent, tag, **kwargs):

    lines = []
    indent = "    "

    # добавляем завершение заголовка функции
    if not kwargs["wasReturn"]:
        valStr = "):\n"
        if tag=="methodbody":
            valStr = indent + valStr
        lines.append(valStr)
        
    # разбираем содержимое
    if tag in ["block","testblock"]: # не добавляем выравнивание
        indent = ""
    if tag=="methodbody": # удваиваем выравнивание
        indent = indent + indent
    for ln in cellContent:
        lines.append(indent+ln) # копируем строки, добавляя выравнивание если нужно

    # результат
    return ( "test" if tag in ["testbody","testblock"] else "mod", lines )

#### Tests

In [ ]:
def test_procBodyTag_smoke():

    assert lit.procBodyTag(
        [
            "print(123)\n",
            "print(4)"
        ],
        tag="body",
        wasReturn=True
    ) == (
        "mod",
        [
            "    print(123)\n",
            "    print(4)"
        ]
    )

In [ ]:
def test_procBodyTag_testbody_smoke():

    assert lit.procBodyTag(
        [
            "print(123)\n",
            "print(4)"
        ],
        tag="testbody",
        wasReturn=True
    ) == (
        "test",
        [
            "    print(123)\n",
            "    print(4)"
        ]
    )

In [ ]:
def test_procBodyTag_no_return():

    assert lit.procBodyTag(
        [
            "print(123)\n",
            "print(4)"
        ],
        tag="body",
        wasReturn=False
    ) == (
        "mod",
        [
            "):\n",
            "    print(123)\n",
            "    print(4)"
        ]
    )

### genAsmNotebook

Функция генерации ASM ноутбука по литературному

* на вход подается имя литературного ноутбука
* результат - имя ASM ноутбука (скорее - для отладки: = лит_asm.ipynb)

В качестве побочного результата - сам ноутбук.

Суть обработки настроена в этом словаре

In [ ]:
tag2Func = {
    "function": procFuncTag,
    "param": procParamTag,
    "testparam": procParamTag,    
    "return": procReturnTag,
    "body": procBodyTag,
    "testbody": procBodyTag,
    "testdef": procFuncTag,
    "block": procBodyTag,
    "testblock": procBodyTag,
    "class": procFuncTag,
    "method": procFuncTag,
    "methodbody": procBodyTag,
}

**Контекст**

Для некоторых конструкций (заголовок теста, опциональность тэга return) необходимо в вызовы procXXX() передавать "контекст".

Пока в словаре контекста содержится

* func: имя текущей обрабатываемой функции
* isMethod: обрабатываем метод класса (для выравнивания закрывающей скобки списка параметров ) 
* wasReturn: наличие тэга return для обрабатываемой функции
    * сбрасывается во время обработки тэгов function или testdef 

#### Source

In [ ]:
import json, os

def genAsmNotebook(litNb):

    with open(litNb,"r") as fp:
        js = json.load(fp)

    cells = js["cells"] # больше нам ничего не нужно - массив ячеек
    asmCells = []
    curFuncName = ""
    wasReturn = True # с таким значением не будет происходить генерации лишнего для блоков кода (типа import)        
    isMethod = False
    for cell in cells:
        if "tags" not in cell["metadata"]: # ячейки без тэгов пропускаем
            continue
        for tag in cell["metadata"]["tags"]: # пока нет смысла давать ячейкам более одного тэга (видимо), но все же цикл...
            if tag=="function" or tag=="method": # запомним в контексте имя функции
                curFuncName = _getFuncName(cell["source"])
                if tag=="function": # обрабатываем не метод
                    isMethod = False
                else:
                    isMethod = True
            if tag in ["function","method","testdef"]: # сбросим информацию о наличии тэга return
                wasReturn = False    
            if tag=="return": # запомним в контексте наличие return
                wasReturn = True
            if tag in ["test","mod","mdef"] or tag.find("mac.")==0: # пробрасываем ячейку как есть в asm ноутбук
                asmCells.append(cell)
            if tag in tag2Func.keys():
                cellTag,cellCode = tag2Func[tag](cell["source"],tag,func=curFuncName,wasReturn=wasReturn,isMethod=isMethod)
                newCell = {
                   "cell_type": "code",
                   # "execution_count": null,
                   "metadata": {
                        "editable": True,
                        "tags": [ cellTag ],
                   },
                   "outputs": [],
                   "source": cellCode
                }
                asmCells.append(newCell)
                if tag in ["body","testbody","methodbody"]: # обработали тело, надо сбросить необходимость добаления ):
                    wasReturn = True

    # сохраняем результат
    dirName,fileName = os.path.split(litNb)
    baseName = fileName.split(".ipynb")[0]
    resName = baseName+"_asm"
    resFile = os.path.join(dirName,resName+".ipynb")

    js["cells"] = asmCells
    with open(resFile,"w") as fp:
        json.dump(js,fp)

    return resFile

#### Tests

Тестов на эту функцию особо не придмаешь - сложно получается (может, потом).

Пока здесь просто вызов, чтобы можно было отлаживаться прям в этом ноутбуке, и чтобы тест порождал файл.

In [ ]:
def test_genAsmNotebook_smoke():

    assert lit.genAsmNotebook("../Examples/lit_smoke.ipynb")=="../Examples/lit_smoke_asm.ipynb"

### genFuncSpec

Функция генерации спецификации функции литературного ноутбука

* на вход подается имя литературного ноутбука и имя функции
* результат - строка спецификации для генерации кода функции роботом

Пока в коде пока вырезал обработку методов ...

#### Source

In [ ]:
def genFuncSpec(litNb,fName,addTask=True):

    with open(litNb,"r") as fp:
        js = json.load(fp)

    cells = js["cells"] # больше нам ничего не нужно - массив ячеек
    lnList = []
    parList = []
    retList = []
    testList = []
    docStr = ""
    toBreak = False
    for cell in cells:
        if toBreak:
            break
        if "tags" not in cell["metadata"]: # ячейки без тэгов пропускаем
            continue
        for tag in cell["metadata"]["tags"]: # пока нет смысла давать ячейкам более одного тэга (видимо), но все же цикл...
            if tag in ["function","class"]:
                if _getFuncName(cell["source"])==fName: # наша функция
                    lnList.append(f"функция {fName}:\n")
                    continue
                else: # другая
                    if lnList: # нашу функцию уже обработали
                        toBreak = True
            if lnList: # обрабатываем описание нашей функции
                if tag=="doc":
                    lnList += ["".join(cell["source"])]
                if tag=="param":
                    if not parList:
                        parList.append(["Функция имеет следующие параметры:"])
                    parList.append([ "* " + ln for ln in cell["source"]])
                if tag=="return":
                    if not retList:
                        retList.append(["Результатами функции должно быть:"])
                    retList.append([ "* " + ln for ln in cell["source"]])
                if tag=="testbody":
                    if not testList:
                        testList.append(["Примеры использования (юнит-тесты - должны выполняться успешно):"])
                    testList.append(cell["source"]) # список - тест может быть большим
                    testList.append([""]) # разделитель
                    
    resStr = "\n".join(lnList)
    if parList:
        resStr += "\n\n" + "\n".join(["\n".join(pEl) for pEl in parList])
    if retList:
        resStr += "\n\n" + "\n".join(["\n".join(rEl) for rEl in retList])
    if testList:
        resStr += "\n\n" + "\n\n".join(["".join(tEl) for tEl in testList])

    if addTask:
        resStr += "\nмне нужен код функции на python"
        
    return resStr